# ONLOAD cells are called at beginning of the report
## Available global variables:
* es: elasticsearch connection
* replacementHT: a dictionary of replacement tags
* report: the report object
* params: the parameters (interval are plits in two name_start + name_end

In [ ]:
#@ONLOAD

replacementHT["Author"]="Arnaud Marchand"
replacementHT["GenerationDate"]=datetime.datetime.now().strftime("%d/%b/%y %H:%M")
replacementHT["Start"]=replacementHT["param2_start"].strftime("%d%b%y")
replacementHT["End"]=replacementHT["param2_end"].strftime("%d%b%y")

In [ ]:
#@ONLOAD

replacementHT["Apps"]=es_helper.elastic_to_dataframe(es,"nyx_app").shape[0]
replacementHT["Reports"]=es_helper.elastic_to_dataframe(es,"nyx_reportdef").shape[0]
replacementHT["Privileges"]=es_helper.elastic_to_dataframe(es,"nyx_privilege").shape[0]
replacementHT["Users"]=es_helper.elastic_to_dataframe(es,"nyx_user").shape[0]

# TEXT

In [ ]:
#@PARAGRAPH=TextParagraph

paragraph.text="Fair trade is an institutional arrangement designed to help producers in developing countries achieve better trading conditions. Members of the fair trade movement advocate the payment of higher prices to exporters, as well as improved social and environmental standards."


In [ ]:
#@PARAGRAPH=DynamicTable

df_privileges=es_helper.elastic_to_dataframe(es,"nyx_privilege")
df_privileges=df_privileges[["_id","name","value"]]
df_privileges.columns=["ID","Name","Value"]
create_table(paragraph,df_privileges, 'Grid Table 4 Accent 5')

#df_privileges=es_helper.elastic_to_dataframe(es,"nyx_user")
#df_privileges=df_privileges[["_id","name","value"]]
#df_privileges.columns=["ID","Name","Value"]
#create_table(paragraph,df_privileges, 'Grid Table 4 Accent 5')

# Charts

In [ ]:
#@PARAGRAPH=PieChart

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Frogs2', 'Hogs', 'Dogs', 'Logs'
sizes = [15, 30, 45, 10]
explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',shadow=False, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.tight_layout()
plt.savefig("./tmp/piechart.png", dpi = 600)

run = paragraph.add_run()
run.add_picture("./tmp/piechart.png", width=Inches(5.6))

In [ ]:
#@PARAGRAPH=BarChart

objects = ('Python', 'C++', 'Java', 'Perl', 'Scala', 'Lisp')
y_pos = np.arange(len(objects))
performance = [10,8,6,4,2,1]

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Usage')
plt.title('Programming language usage')

plt.tight_layout()
plt.savefig("./tmp/barchart.png", dpi = 600)

run = paragraph.add_run()
run.add_picture("./tmp/barchart.png", width=Inches(3.6))

# BROL

In [ ]:
from elasticsearch import Elasticsearch as ES, RequestsHttpConnection as RC
import collections
import datetime
from elastic_helper import es_helper

host_params="http://esnodebal:9200"
es = ES(hosts=[host_params])

es.info()
replacementHT={}